<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/ID3DT02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install necessary packages
!pip install numpy
!pip install python-javabridge
!pip install python-weka-wrapper3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# download datasets
!wget https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/iris.arff
!wget https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/anneal.arff
!wget https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/bolts.arff
!wget https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/bodyfat.arff


--2022-09-20 08:57:11--  https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/iris.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7486 (7.3K) [text/plain]
Saving to: ‘iris.arff’

iris.arff           100%[===================>]   7.31K  --.-KB/s    in 0s      

2022-09-20 08:57:11 (57.8 MB/s) - ‘iris.arff’ saved [7486/7486]

--2022-09-20 08:57:12--  https://raw.githubusercontent.com/fracpete/python-weka-wrapper3-examples/master/src/wekaexamples/data/anneal.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, await

In [3]:
# start JVM
import weka.core.jvm as jvm

jvm.start()


DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/usr/local/lib/python3.7/dist-packages/javabridge/jars/rhino-1.7R4.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/runnablequeue.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/cpython.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/weka.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/python-weka-wrapper.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [4]:
import os
import tempfile
import traceback
import weka.core.converters as converters
from weka.classifiers import Classifier
from weka.experiments import SimpleCrossValidationExperiment, SimpleRandomSplitExperiment, Tester, ResultMatrix

datasets = ["iris.arff", "anneal.arff"]
classifiers = [Classifier("weka.classifiers.rules.ZeroR"), Classifier("weka.classifiers.trees.J48")]
outfile = tempfile.gettempdir() + os.sep + "results-cv.arff"
exp = SimpleCrossValidationExperiment(
    classification=True,
    runs=10,
    folds=10,
    datasets=datasets,
    classifiers=classifiers,
    result=outfile)
exp.setup()
exp.run()

# evaluate
loader = converters.loader_for_file(outfile)
data = loader.load_file(outfile)
matrix = ResultMatrix("weka.experiment.ResultMatrixPlainText")
tester = Tester("weka.experiment.PairedCorrectedTTester")
tester.resultmatrix = matrix
comparison_col = data.attribute_by_name("Percent_correct").index
tester.instances = data
print(tester.header(comparison_col))
print(tester.multi_resultset_full(0, comparison_col))

# random split + regression
datasets = ["bolts.arff", "bodyfat.arff"]
classifiers = [
    Classifier("weka.classifiers.rules.ZeroR"),
    Classifier("weka.classifiers.functions.LinearRegression")
]
outfile = tempfile.gettempdir() + os.sep + "results-rs.arff"
exp = SimpleRandomSplitExperiment(
    classification=False,
    runs=10,
    percentage=66.6,
    preserve_order=False,
    datasets=datasets,
    classifiers=classifiers,
    result=outfile)
exp.setup()
exp.run()

# evaluate
loader = converters.loader_for_file(outfile)
data = loader.load_file(outfile)
matrix = ResultMatrix("weka.experiment.ResultMatrixPlainText")
tester = Tester("weka.experiment.PairedCorrectedTTester")
tester.resultmatrix = matrix
comparison_col = data.attribute_by_name("Correlation_coefficient").index
tester.instances = data
print(tester.header(comparison_col))
print(tester.multi_resultset_full(0, comparison_col))


Tester:     weka.experiment.PairedCorrectedTTester -G 4,5,6 -D 1 -R 2 -S 0.05 -result-matrix "weka.experiment.ResultMatrixPlainText -mean-prec 2 -stddev-prec 5 -col-name-width 0 -row-name-width 25 -mean-width 0 -stddev-width 0 -sig-width 0 -count-width 5 -enum-col-names -enum-row-names"
Analysing:  Percent_correct
Datasets:   2
Resultsets: 2
Confidence: 0.05 (two tailed)
Sorted by:  -
Date:       9/20/22, 8:57 AM


Dataset                   (1)          | (2)      
--------------------------------------------------
(1)                      (100)   33.33 |   94.73 v
(2)                      (100)   76.17 |   98.57 v
--------------------------------------------------
                               (v/ /*) |   (2/0/0)


Key:
(1) rules.ZeroR '' 48055541465867954
(2) trees.J48 '-C 0.25 -M 2' -217733168393644444

Tester:     weka.experiment.PairedCorrectedTTester -G 3,4,5 -D 1 -R 2 -S 0.05 -result-matrix "weka.experiment.ResultMatrixPlainText -mean-prec 2 -stddev-prec 5 -col-name-width 0 -ro

In [5]:
# stop JVM
jvm.stop()